In [1]:
pip install -U scikit-learn 

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Openclassrooms/P7 scoring/git_collab_P7/git_collab_P7')

import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE 
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

import sklearn


import joblib

In [ ]:
from My_Fonctions.fonctions import *

In [ ]:
data = "/content/drive/MyDrive/Openclassrooms/P7 scoring/data/"

## pre processing du data set

Fonction réalisé à partir du kernel Kaggle : https://www.kaggle.com/jsaguiar/lightgbm-with-simple-features

In [ ]:
df_data = preproc(data,num_row=None)

Train samples: 307511
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 30s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 27s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 16s
Installments payments df shape: (339587, 26)
Process installments payments - done in 39s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 25s


Sauvegarde du data set

In [ ]:
joblib.dump(df_data, data + 'df_data.joblib')

['/content/drive/MyDrive/Openclassrooms/P7 scoring/data/df_data.joblib']

In [ ]:
df_data.shape

(356251, 798)

In [ ]:
df_data['TARGET'].value_counts()

0.0    282682
1.0     24825
Name: TARGET, dtype: int64

In [ ]:
df_data['TARGET'].isna().sum() 

48744

In [ ]:
df_data.dtypes

index                                 int64
SK_ID_CURR                            int64
TARGET                              float64
CODE_GENDER                           int64
FLAG_OWN_CAR                          int64
                                     ...   
CC_NAME_CONTRACT_STATUS_nan_MAX     float64
CC_NAME_CONTRACT_STATUS_nan_MEAN    float64
CC_NAME_CONTRACT_STATUS_nan_SUM     float64
CC_NAME_CONTRACT_STATUS_nan_VAR     float64
CC_COUNT                            float64
Length: 798, dtype: object

# restauration du dataset

In [ ]:
df_data = joblib.load(data + 'df_data.joblib')

supression des colonnes non necessaires

In [ ]:
df_data.drop(columns=['SK_ID_CURR','index'],inplace=True)

In [ ]:
df_data.dtypes

TARGET                              float64
CODE_GENDER                           int64
FLAG_OWN_CAR                          int64
FLAG_OWN_REALTY                       int64
CNT_CHILDREN                          int64
                                     ...   
CC_NAME_CONTRACT_STATUS_nan_MAX     float64
CC_NAME_CONTRACT_STATUS_nan_MEAN    float64
CC_NAME_CONTRACT_STATUS_nan_SUM     float64
CC_NAME_CONTRACT_STATUS_nan_VAR     float64
CC_COUNT                            float64
Length: 796, dtype: object

Séparation du data set train / test

In [ ]:
df_data_train = df_data[df_data["TARGET"].notnull() ]

In [ ]:
df_data_test =  df_data[df_data["TARGET"].isnull() ]

# Péparation du data set train

Le jeux de donnée est désiquilibre.
Nous utilison l'option de stratify lors de la selection des du train pour conserver le meme désequilibre

In [ ]:
train, test = train_test_split(df_data_train, test_size=0.2, stratify=df_data_train['TARGET']) 

In [ ]:
df_data['TARGET'].value_counts()

0.0    282682
1.0     24825
Name: TARGET, dtype: int64

In [ ]:
train['TARGET'].value_counts()

0.0    226145
1.0     19860
Name: TARGET, dtype: int64

In [ ]:
test['TARGET'].value_counts()

0.0    56537
1.0     4965
Name: TARGET, dtype: int64

Le ration de 10% est conservé entre les target 1 et 0

Reduction du jeux de donnée qui est trop important pour tester plusieur modele et methode de gestion des déséquilibres de données

In [ ]:
y_train = train["TARGET"]
X_train = train.drop(["TARGET"],axis=1)
y_test = test["TARGET"]
X_test = test.drop(["TARGET"],axis=1)

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.9, random_state=0)
sss.get_n_splits(X_train, y_train)

for train_index, test_index in sss.split(X_train, y_train):
    
    #selection d'un échantillon aléatoire stratifié pour respecter les proportions
    
    X_sample = X_train.iloc[train_index,:] 
    y_sample = y_train.iloc[train_index]

# fonction de cout utilisé

La librairie imbleanr permet d'afficher des metrics specifique au dataset désequilibré tel que le score F1 et la moyenne geometrique

https://imbalanced-learn.org/stable/

# ACP

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
scaler = StandardScaler()

In [ ]:
acp = PCA(n_components=0.99)

In [ ]:
pipe_acp = make_pipeline(imp,scaler,acp )

In [ ]:
x_train_acp = pipe_acp.fit_transform(X_sample)

In [ ]:
X_sample.shape

(24600, 795)

In [ ]:
x_train_acp.shape

(24600, 470)

### Variance

In [ ]:
selector = VarianceThreshold(threshold=(.8 * (1 - .8)))

In [ ]:
pipe_variance = make_pipeline(imp,scaler,selector )

In [ ]:
x_train_variance = pipe_variance.fit_transform(X_sample)

In [ ]:
x_train_variance.shape

(24600, 738)

# test de modele

In [ ]:
lr = LogisticRegression()

In [ ]:
pipe_reg = make_pipeline(imp,scaler,lr )

In [ ]:
pipe_reg.fit(X_sample, y_sample)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_pred = pipe_reg.predict(X_test)

In [ ]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      0.99      0.05      0.96      0.22      0.05     56537
        1.0       0.35      0.05      0.99      0.08      0.22      0.04      4965

avg / total       0.88      0.92      0.12      0.89      0.22      0.05     61502



In [ ]:
rdm = RandomForestClassifier()

In [ ]:
pipe_rdm = make_pipeline(imp,rdm )

In [ ]:
pipe_rdm.fit(X_sample, y_sample)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
y_pred = pipe_rdm.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      1.00      0.00      0.96      0.02      0.00     56537
        1.0       1.00      0.00      1.00      0.00      0.02      0.00      4965

avg / total       0.93      0.92      0.08      0.88      0.02      0.00     61502



In [ ]:
lgbm = LGBMClassifier()

In [ ]:
pipe_lgbm = make_pipeline(imp,lgbm )

In [ ]:
pipe_lgbm.fit(X_sample, y_sample)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('lgbmclassifier', LGBMClassifier())])

In [ ]:
y_pred = pipe_lgbm.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      1.00      0.04      0.96      0.20      0.04     56537
        1.0       0.44      0.04      1.00      0.07      0.20      0.03      4965

avg / total       0.88      0.92      0.12      0.89      0.20      0.04     61502



## Exploitation des methodes de gestion des data set déséquilibré

### sur échantillonnage

Regression

In [ ]:
rOs = RandomOverSampler()
pipe_norm_lr = make_pipeline(imp,scaler)
X_train_norm = pipe_norm_lr.fit_transform(X_sample)
X_test_norm = pipe_norm_lr.transform(X_test)
X_ro, y_ro = rOs.fit_resample(X_train_norm, y_sample)
lr.fit(X_ro, y_ro)
y_pred = lr.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.96      0.73      0.63      0.83      0.68      0.46     56537
        1.0       0.17      0.63      0.73      0.27      0.68      0.45      4965

avg / total       0.89      0.72      0.64      0.78      0.68      0.46     61502



In [ ]:
print ( ' Nb feature avant', y_sample.shape , 'nb feature apres' ,y_ro.shape)

 Nb feature avant (24600,) nb feature apres (45228,)


In [ ]:
print ( 'répartition ' , y_ro.value_counts(normalize=True) * 100 )

répartition  1.0    50.0
0.0    50.0
Name: TARGET, dtype: float64


Random

In [ ]:
pipe_norm_rdm = make_pipeline(imp)
X_train_norm = pipe_norm_rdm.fit_transform(X_sample)
X_test_norm = pipe_norm_rdm.transform(X_test)
X_ro, y_ro = rOs.fit_resample(X_train_norm, y_sample)
rdm.fit(X_ro, y_ro)
y_pred = rdm.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      1.00      0.01      0.96      0.08      0.01     56537
        1.0       0.54      0.01      1.00      0.01      0.08      0.01      4965

avg / total       0.89      0.92      0.09      0.88      0.08      0.01     61502



lgbm

In [ ]:
lgbm.fit(X_ro, y_ro)
y_pred = lgbm.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.95      0.86      0.46      0.90      0.63      0.41     56537
        1.0       0.22      0.46      0.86      0.30      0.63      0.38      4965

avg / total       0.89      0.83      0.49      0.85      0.63      0.40     61502



### Sous echantillonnage

In [ ]:
rUs = RandomUnderSampler()

In [ ]:
pipe_norm_lr = make_pipeline(imp,scaler)
X_train_norm = pipe_norm_lr.fit_transform(X_sample)
X_test_norm = pipe_norm_lr.transform(X_test)
X_ru, y_ru = rUs.fit_resample(X_train_norm, y_sample)
lr.fit(X_ru, y_ru)
y_pred = lr.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.96      0.67      0.66      0.79      0.66      0.44     56537
        1.0       0.15      0.66      0.67      0.24      0.66      0.44      4965

avg / total       0.89      0.67      0.66      0.74      0.66      0.44     61502



In [ ]:
print ( ' Nb feature avant', y_sample.shape , 'nb feature apres' ,y_ru.shape)

 Nb feature avant (24600,) nb feature apres (3972,)


In [ ]:
print ( 'répartition ' , y_ru.value_counts(normalize=True) * 100 )

répartition  1.0    50.0
0.0    50.0
Name: TARGET, dtype: float64


Random

In [ ]:
pipe_norm_rdm = make_pipeline(imp)
X_train_norm = pipe_norm_rdm.fit_transform(X_sample)
X_test_norm = pipe_norm_rdm.transform(X_test)
X_ru, y_ru = rUs.fit_resample(X_train_norm, y_sample)
rdm.fit(X_ru, y_ru)
y_pred = rdm.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.96      0.68      0.64      0.80      0.66      0.44     56537
        1.0       0.15      0.64      0.68      0.24      0.66      0.44      4965

avg / total       0.89      0.68      0.64      0.75      0.66      0.44     61502



lgbm

In [ ]:
lgbm.fit(X_ru, y_ru)
y_pred = lgbm.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.96      0.68      0.69      0.80      0.69      0.47     56537
        1.0       0.16      0.69      0.68      0.26      0.69      0.47      4965

avg / total       0.90      0.68      0.69      0.76      0.69      0.47     61502



### SMOT

LR

In [ ]:
smo = SMOTE()

In [ ]:
pipe_norm_lr = make_pipeline(imp,scaler)
X_train_norm = pipe_norm_lr.fit_transform(X_sample)
X_test_norm = pipe_norm_lr.transform(X_test)
X_sm, y_sm = smo.fit_resample(X_train_norm, y_sample)
lr.fit(X_sm, y_sm)
y_pred = lr.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.95      0.74      0.58      0.83      0.65      0.43     56537
        1.0       0.16      0.58      0.74      0.25      0.65      0.42      4965

avg / total       0.89      0.72      0.59      0.78      0.65      0.43     61502



In [ ]:
print ( ' Nb feature avant', y_sample.shape , 'nb feature apres' ,y_sm.shape)

 Nb feature avant (24600,) nb feature apres (45228,)


In [ ]:
y_sm.value_counts(normalize=True) * 100

1.0    50.0
0.0    50.0
Name: TARGET, dtype: float64

Random

In [ ]:
pipe_norm_rdm = make_pipeline(imp)
X_train_norm = pipe_norm_rdm.fit_transform(X_sample)
X_test_norm = pipe_norm_rdm.transform(X_test)
X_sm, y_sm = smo.fit_resample(X_train_norm, y_sample)
rdm.fit(X_sm, y_sm)
y_pred = rdm.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      1.00      0.00      0.96      0.01      0.00     56537
        1.0       0.14      0.00      1.00      0.00      0.01      0.00      4965

avg / total       0.86      0.92      0.08      0.88      0.01      0.00     61502



LGBM

In [ ]:
lgbm.fit(X_sm, y_sm)
y_pred = lgbm.predict(X_test_norm)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      1.00      0.03      0.96      0.18      0.04     56537
        1.0       0.46      0.03      1.00      0.06      0.18      0.03      4965

avg / total       0.88      0.92      0.11      0.89      0.18      0.04     61502

